## CMSEtech Webscraper

### Extract CMSE classes information

In [4]:
from bs4 import BeautifulSoup
import re
import pandas as pd

In [95]:
with open('./Manage Classes.html') as f:
    html_doc = f.read()

soup = BeautifulSoup(html_doc, 'html.parser')
tables = soup.findAll("table", {"class": "ps_grid-flex"})
tab = tables[1]
trs = tab.findAll('tr')


In [137]:
df = pd.DataFrame()  # create dataframe to store values
col_names = ['class_name', 'units', 'section', 'time', 'duration','instructor']
values = []
for tr in trs[1:]:
    tds = tr.find_all("td")
    vals = []
    for td in tds:
        #print(td.text)
        vals.append(td.text.strip().split('\n\n\n\n\n'))
        val = [item for sublist in vals for item in sublist]
    #vals.append(tds[1].text.strip())
    values.append(val)

In [139]:
for i, col_name in enumerate(col_names):
    df[col_name] = [row[i] for row in values]
df

,class_name,units,section,time,duration,instructor
0,CMSE 201: Computational Modeling and Data Anal...,Lecture (4.00 units) Wait List,Section 002 / Class Nbr 14273 / Regular Academ...,Mon Wed : 10:20 AM-12:10 PM,08/28/2023 - 12/10/2023,To Be Announced
1,CMSE 201: Computational Modeling and Data Anal...,Lecture (4.00 units) Wait List,Section 003 / Class Nbr 14274 / Regular Academ...,Mon Wed : 12:40 PM-2:30 PM,08/28/2023 - 12/10/2023,To Be Announced
2,CMSE 201: Computational Modeling and Data Anal...,Lecture (4.00 units) Wait List,Section 004 / Class Nbr 14275 / Regular Academ...,Mon Wed : 3:00 PM-4:50 PM,08/28/2023 - 12/10/2023,To Be Announced
3,CMSE 201: Computational Modeling and Data Anal...,Lecture (4.00 units) Open,Section 005 / Class Nbr 14276 / Regular Academ...,Tue Thu : 8:00 AM-9:50 AM,08/28/2023 - 12/10/2023,To Be Announced
4,CMSE 201: Computational Modeling and Data Anal...,Lecture (4.00 units) Wait List,Section 006 / Class Nbr 14277 / Regular Academ...,Tue Thu : 10:20 AM-12:10 PM,08/28/2023 - 12/10/2023,To Be Announced
5,CMSE 201: Computational Modeling and Data Anal...,Lecture (4.00 units) Wait List,Section 007 / Class Nbr 14278 / Regular Academ...,Tue Thu : 12:40 PM-2:30 PM,08/28/2023 - 12/10/2023,To Be Announced
6,CMSE 202: Computational Modeling and Data Anal...,Lecture (4.00 units) Wait List,Section 001 / Class Nbr 14279 / Regular Academ...,Mon Wed : 10:20 AM-12:10 PM,08/28/2023 - 12/10/2023,To Be Announced
7,CMSE 202: Computational Modeling and Data Anal...,Lecture (4.00 units) Wait List,Section 002 / Class Nbr 14280 / Regular Academ...,Mon Wed : 3:00 PM-4:50 PM,08/28/2023 - 12/10/2023,To Be Announced
8,CMSE 202: Computational Modeling and Data Anal...,Lecture (4.00 units) Open,Section 003 / Class Nbr 14281 / Regular Academ...,Tue Thu : 3:00 PM-4:50 PM,08/28/2023 - 12/10/2023,To Be Announced
9,CMSE 314: Matrix Algebra with Computational Ap...,Lecture (3.00 units) Wait List,Section 001 / Class Nbr 15401 / Regular Academ...,Tue Thu : 10:20 AM-11:40 AM,08/28/2023 - 12/10/2023,To Be Announced
